# Date Night Movie

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting


In [16]:
import os
import pandas as pd

##### Read in the movie data: `pd.read_table`

In [31]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames, encoding='unicode_escape')
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames, encoding='unicode_escape')
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames, encoding='unicode_escape')

    return users, ratings, movies

In [32]:
users, ratings, movies = get_movie_data()

In [33]:
print (users.head())

   user_id gender  age  occupation    zip
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455


In [34]:
print (ratings.head())

   user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291


In [35]:
print (movies.head())

   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [37]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

0    Toy Story
1         1995
dtype: object

In [38]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [39]:
print (movies.head())

   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   year                  short_title  
0  1995                    Toy Story  
1  1995                      Jumanji  
2  1995             Grumpier Old Men  
3  1995            Waiting to Exhale  
4  1995  Father of the Bride Part II  


##### Join the tables with `pd.merge` (20 pts)
###### 1) join movies and ratings table based on movie_id
###### 2) join result of above join with users table on user_id

In [40]:
# 1) join movies and ratings table based on movie_id
movies_ratings = pd.merge(movies,ratings,on = 'movie_id', how = 'inner')
movies_ratings.head()

,movie_id,title,genres,year,short_title,user_id,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,10,5,978226474


In [41]:
#2) join result of above join with users table on user_id
df = pd.merge(movies_ratings,users,on = 'user_id', how = 'inner')
df.head()

,movie_id,title,genres,year,short_title,user_id,rating,timestamp,gender,age,occupation,zip
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1995,Pocahontas,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1995,Apollo 13,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1977,Star Wars: Episode IV - A New Hope,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1993,Schindler's List,1,5,978824195,F,1,10,48067


##### What's the highest rated movie? (20 pts))
##### 1) Group movies by their ids and count the number of times each movies is rated by users. Save it as times_rated data frame.
##### 2) Group movies by their ids and calcultae average rating for each movies based on users rating. Save it as average_rating data frame.
##### 3) Select top 200 movies with most ratings from times_rated data frame and merge it with average_rating data frame. Save it as top_200_rated_movies_avg_rating data frame.   
##### 4) Sort top_200_rated_movies_avg_rating data frame based on values in rating column. Show the top 5 movies.

In [42]:
# 1) Group movies by their ids and count the number of times each movies is rated by users. Save it as times_rated data frame.
times_rated = df.groupby(['movie_id','title']).count()['rating'].sort_values(ascending = False).reset_index()
times_rated.columns = ['movie_id', 'title', 'times_rated']
times_rated.head()

,movie_id,title,times_rated
0,2858,American Beauty (1999),3428
1,260,Star Wars: Episode IV - A New Hope (1977),2991
2,1196,Star Wars: Episode V - The Empire Strikes Back...,2990
3,1210,Star Wars: Episode VI - Return of the Jedi (1983),2883
4,480,Jurassic Park (1993),2672


In [43]:
# 2) Group movies by their ids and calcultae average rating for each movies based on users rating. Save it as average_rating data frame.
average_rating = df.groupby(['movie_id','title']).mean()['rating'].sort_values(ascending = False).reset_index()
average_rating.head()

,movie_id,title,rating
0,989,Schlafes Bruder (Brother of Sleep) (1995),5.0
1,3881,Bittersweet Motel (2000),5.0
2,1830,Follow the Bitch (1998),5.0
3,3382,Song of Freedom (1936),5.0
4,787,"Gate of Heavenly Peace, The (1995)",5.0


In [44]:
# 3) Select top 200 movies with most ratings from times_rated data frame and merge it with average_rating data frame.
# Save it as top_200_rated_movies_avg_rating data frame.
top_200_rated_movies_avg_rating = pd.merge(times_rated[0:200],average_rating,on = ['movie_id','title'], how ='inner')


In [45]:
# 4) Sort top_200_rated_movies_avg_rating data frame based on values in rating column. Show the top 5 movies.
top_200_rated_movies_avg_rating.sort_values(by=['rating'], ascending = False).head()

,movie_id,title,times_rated,rating
23,318,"Shawshank Redemption, The (1994)",2227,4.554558
24,858,"Godfather, The (1972)",2223,4.524966
38,50,"Usual Suspects, The (1995)",1783,4.517106
17,527,Schindler's List (1993),2304,4.510417
11,1198,Raiders of the Lost Ark (1981),2514,4.477725


#### Response:
##### "Shawshank Redemption, The (1994)" has the highest average rate among top 200 movies with most ratings.
##### "American Beauty (1999)" is the movie rated by most users and has highest number of ratings.

#### What is a good rated movie for date night? (60 pts)

- Hint - highly rated movie by 
    - both partners (might be the same gender or not),
    - based on genre preferences,
    - age group can also be combined
    


#### Response:
##### For this I use the codes I wrote for previous question to define a function. Then I used this function, to come up with the most popular movie for various genres cosidering gender and age group. 
##### This function takes four inputs and returns one output:
##### Inputs:
###### - age: should be one of the values listed in the following list: [ 1, 50, 25, 35, 18, 45, 56]. If, there is no age preference, then it should be provided as "None".  
###### - gender: should be either "F" or "M". If, there is no gender preference, then it should be provided as "None".  
###### - genre: should be one of the strings listed in the following list: ['Animation', 'Drama', 'Action', "Children's", 'Crime', 'Musical','Adventure', 'Comedy', 'Romance', 'Thriller', 'Western','Documentary', 'Sci-Fi', 'Horror', 'Film-Noir', 'Mystery', 'War','Fantasy']. If, there is no genre preference, then it should be provided as "None".   
##### Output: 
###### - A list with top 10 highly rated movies based on selection criteria.


In [46]:
def popular_movies_for_certain_group (df, age, gender, genre):    
    if age is None:
        df = df
    else:
        df = df[df['age'] == age]
    if gender is None:
        df = df
    else:
        df = df[df['gender'] == gender]
    if genre is None:
        df = df
    else:
        df = df[df['broad_genres'] == genre]
    times_rated = df.groupby(['movie_id','title']).count()['rating'].sort_values(ascending = False).reset_index()
    times_rated.columns = ['movie_id', 'title', 'times_rated']
    average_rating = df.groupby(['movie_id','title']).mean()['rating'].sort_values(ascending = False).reset_index()
    top_200_rated_movies_avg_rating = pd.merge(times_rated[0:200],average_rating,on = ['movie_id','title'], how ='inner')
    return list(top_200_rated_movies_avg_rating.sort_values(by=['rating'], ascending = False)['title'].head(10))

#### genres column in data frame is too specific. I made it a bit broader by selecting the genre that movie is most known for. This would be generally a string before "|" character.

In [47]:
df['broad_genres'] =  [item.split('|')[0] for item in df['genres']]
df['broad_genres'].unique()

array(['Animation', 'Drama', 'Action', "Children's", 'Crime', 'Musical',
       'Adventure', 'Comedy', 'Romance', 'Thriller', 'Western',
       'Documentary', 'Sci-Fi', 'Horror', 'Film-Noir', 'Mystery', 'War',
       'Fantasy'], dtype=object)

### Sample response for female couple with age of 45 that prefere Drama genre: 

In [48]:
popular_movies_for_certain_group (df, 45, 'F', 'Drama')

["Schindler's List (1993)",
 'Lone Star (1996)',
 'Shawshank Redemption, The (1994)',
 "It's a Wonderful Life (1946)",
 'Gone with the Wind (1939)',
 'Streetcar Named Desire, A (1951)',
 'To Kill a Mockingbird (1962)',
 'Amadeus (1984)',
 'Secrets & Lies (1996)',
 'Some Folks Call It a Sling Blade (1993)']

### Response: Loop over all possible combinations, and select the one which is most repeated among top 10s in all catagories: 
#### - identify all possible combinations:

In [49]:
import itertools
possible_genders = ['F','M',None]
possible_ages = [1, 50, 25, 35, 18, 45, 56,None]
possible_genres = [None, 'Animation', 'Drama', 'Action', "Children's", 'Crime', 'Musical','Adventure', 'Comedy', 'Romance', 'Thriller', 'Western','Documentary', 'Sci-Fi', 'Horror', 'Film-Noir', 'Mystery', 'War','Fantasy']
possible_combos = [possible_ages,possible_genders,possible_genres]
all_possible_combos = list(itertools.product(*possible_combos))

#### use popular_movies_for_certain_group function to pick top 10 movies for all possible criteria

In [50]:
movie_list = []
for item in all_possible_combos:
    movie_list.append(popular_movies_for_certain_group (df, item[0], item[1], item[2]))


#### create a flat list from above list

In [51]:
movie_flat_list =  [item for sublist in movie_list for item in sublist]

### Response:
#### Find top 10 movies that are popular among most groups with various preferences:

In [52]:
ndf = pd.DataFrame()
ndf['movies'] = movie_flat_list
ndf['popular_by_x_category'] = 1
ndf.groupby('movies').count().sort_values(by = 'popular_by_x_category', ascending = False)[:10]

,popular_by_x_category
movies,
Schindler's List (1993),43
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),37
"Godfather, The (1972)",34
Rear Window (1954),34
"Sixth Sense, The (1999)",32
"Shawshank Redemption, The (1994)",32
"Usual Suspects, The (1995)",31
Raiders of the Lost Ark (1981),29
Star Wars: Episode IV - A New Hope (1977),28


### Schindler's List (1993) is among top 10 movies in 43 categories of 456 catagories of audiance evaluated here  

In [53]:
len(all_possible_combos)

456